In [9]:
import pandas as pd
import numpy as np
from copy import copy
import matplotlib.pyplot as plt
import seaborn as sns

## answers
 - 102
 - Facebook_ru.wikipedia.org_desktop_all-agents
 - 54.16127748085961
 - 52.465883293370794

## Read files

In [10]:
file_train = '../data/wikipedia_train.csv'
file_test = '../data/wikipedia_test.csv'

In [11]:
df_train = pd.read_csv(file_train)
df_test = pd.read_csv(file_test)

## Make 'Date' a column

In [12]:
columns_train = list(df_train.columns)
columns_train.pop(0)
columns_test = list(df_test.columns)
columns_test.pop(0)

'Page'

In [13]:
df_train = pd.melt(df_train, id_vars=['Page'], value_vars=columns_train, var_name='Date', value_name='Visits')
df_test = pd.melt(df_test, id_vars=['Page'], value_vars=columns_test, var_name='Date', value_name='Visits')

In [14]:
df_train.head()

,Page,Date,Visits
0,15._November_de.wikipedia.org_desktop_all-agents,2015-07-01,32.0
1,2012_(film)_fr.wikipedia.org_all-access_spider,2015-07-01,2.0
2,2016_FIFA_U-20女子ワールドカップ_ja.wikipedia.org_all-a...,2015-07-01,1.0
3,2016_UEFA_Europa_League_Final_en.wikipedia.org...,2015-07-01,3.0
4,2016_in_video_gaming_en.wikipedia.org_all-acce...,2015-07-01,24.0


In [15]:
df_train.describe()

,Visits
count,385033.000000
mean,821.064911
std,4128.692870
min,0.000000
25%,14.000000
50%,114.000000
75%,609.000000
max,870849.000000


In [16]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401892 entries, 0 to 401891
Data columns (total 3 columns):
Page      401892 non-null object
Date      401892 non-null object
Visits    385033 non-null float64
dtypes: float64(1), object(2)
memory usage: 9.2+ MB


## Create ru_pages  - dataFrame where only russian pages are

In [17]:
import re
def get_language(page):
    pattern = r'[a-z][a-z].wikipedia.org'
    result = re.search(pattern, page)
    if result:
        return result.group(0)[0:2]
    return 'na'

def get_certain_lang_page(page, lan='ru'):
    if get_language(page) == lan:
        return True
    return False

In [18]:
# fill all empty 'visits' to 0
df_train['Page'] = df_train['Page'].fillna(0)

In [19]:
# number of russian pages in the dataset
res = filter(get_certain_lang_page, df_train['Page'])
len(list(res))
res = [x for x in df_train['Page'] if get_certain_lang_page(x, 'ru')]
len(res)

43656

In [20]:
ru = filter(lambda x: get_certain_lang_page(x, 'ru'), df_train['Page'])
list(ru)

['22_апреля_ru.wikipedia.org_all-access_spider',
 '6-я_армия_(Третий_рейх)_ru.wikipedia.org_desktop_all-agents',
 'Apple_ru.wikipedia.org_mobile-web_all-agents',
 'Dishonored_ru.wikipedia.org_all-access_all-agents',
 'Facebook_ru.wikipedia.org_desktop_all-agents',
 'Sukhoi_Superjet_100_ru.wikipedia.org_desktop_all-agents',
 'Victoria’s_Secret_ru.wikipedia.org_all-access_spider',
 'А_зори_здесь_тихие_(фильм,_1972)_ru.wikipedia.org_mobile-web_all-agents',
 'Агенты_«Щ.И.Т.»_ru.wikipedia.org_desktop_all-agents',
 'Атлетико_Мадрид_ru.wikipedia.org_mobile-web_all-agents',
 'Афон_ru.wikipedia.org_all-access_spider',
 'Бадоева,_Жанна_Осиповна_ru.wikipedia.org_all-access_spider',
 'Буркина-Фасо_ru.wikipedia.org_desktop_all-agents',
 'Ван_Гог,_Винсент_ru.wikipedia.org_desktop_all-agents',
 'Варяги_ru.wikipedia.org_mobile-web_all-agents',
 'Вокс-Бурмистрова,_Алиса_Михайловна_ru.wikipedia.org_desktop_all-agents',
 'Волан-де-Морт_ru.wikipedia.org_all-access_spider',
 'Время_в_России_ru.wikipedia.or

In [21]:
pattern = [get_certain_lang_page(x, 'ru') for x in df_train['Page']]
ru_pages = df_train.loc[pattern]
ru_pages.info()
ru_pages.describe()
ru_pages.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43656 entries, 6 to 401686
Data columns (total 3 columns):
Page      43656 non-null object
Date      43656 non-null object
Visits    42851 non-null float64
dtypes: float64(1), object(2)
memory usage: 1.3+ MB


,Page,Date,Visits
6,22_апреля_ru.wikipedia.org_all-access_spider,2015-07-01,18.0
8,6-я_армия_(Третий_рейх)_ru.wikipedia.org_deskt...,2015-07-01,16.0
38,Apple_ru.wikipedia.org_mobile-web_all-agents,2015-07-01,391.0
154,Dishonored_ru.wikipedia.org_all-access_all-agents,2015-07-01,792.0
180,Facebook_ru.wikipedia.org_desktop_all-agents,2015-07-01,1665.0


In [22]:
# what null object in the ru_pages mean
ru_pages.loc[ru_pages['Visits'] == 0].tail()

,Page,Date,Visits
314297,"Каррера,_Массимо_ru.wikipedia.org_mobile-web_a...",2016-05-30,0.0
319931,"Каррера,_Массимо_ru.wikipedia.org_mobile-web_a...",2016-06-05,0.0
320870,"Каррера,_Массимо_ru.wikipedia.org_mobile-web_a...",2016-06-06,0.0
326504,"Каррера,_Массимо_ru.wikipedia.org_mobile-web_a...",2016-06-12,0.0
337821,Убийца_Крок_ru.wikipedia.org_all-access_spider,2016-06-24,0.0


In [23]:
unique_ru_pages = ru_pages['Page'].unique()
len(unique_ru_pages)

102

## Group ru_pages by the 'page' column

In [24]:
ru_pages.groupby(['Page']).mean().sort_values(['Visits'], ascending=False).head()

,Visits
Page,
Facebook_ru.wikipedia.org_desktop_all-agents,2171.240654
"Цветаева,_Марина_Ивановна_ru.wikipedia.org_all-access_all-agents",1873.478972
Звёздные_войны._Эпизод_I:_Скрытая_угроза_ru.wikipedia.org_all-access_all-agents,1856.995327
Марокко_ru.wikipedia.org_all-access_all-agents,1787.502336
Мужской_половой_член_ru.wikipedia.org_all-access_all-agents,1708.794393


In [25]:
# mean value
ru_pages_grouped = ru_pages.groupby(['Page']).mean().sort_values(['Visits'], ascending=False)

In [26]:
last_visits = ru_pages.groupby(['Page']).last()

In [27]:
# last visits
ru_pages_grouped['Last visits'] = last_visits['Visits']
ru_pages_grouped.head()

,Visits,Last visits
Page,,
Facebook_ru.wikipedia.org_desktop_all-agents,2171.240654,2235.0
"Цветаева,_Марина_Ивановна_ru.wikipedia.org_all-access_all-agents",1873.478972,4427.0
Звёздные_войны._Эпизод_I:_Скрытая_угроза_ru.wikipedia.org_all-access_all-agents,1856.995327,504.0
Марокко_ru.wikipedia.org_all-access_all-agents,1787.502336,1471.0
Мужской_половой_член_ru.wikipedia.org_all-access_all-agents,1708.794393,1416.0
